In [ ]:
import cv2
import sys
import os

data_dir = 'C:/Users/kseongwoo/datasets/Polyp/612polyp/gt/'
save_dir = 'C:/Users/kseongwoo/datasets/Polyp/612polyp/label/'
image_list = os.listdir(data_dir)

# image_list.sort(key=lambda fname: int(fname.split('.')[0]))

# with open(save_dir + 'classes.txt', 'w') as sys.stdout:
#     print("polyp")
#     sys.stdout = temp

def convert(size, box): 
    dw = 1./size[0] 
    dh = 1./size[1] 

    x_center = (box[0] + box[2])/2.0 
    y_center = (box[1] + box[3])/2.0 
    w = box[2] - box[0] 
    h = box[3] - box[1] 

    x_center *= dw 
    y_center *= dh 
    
    w *= dw 
    h *= dh 

    return (x_center,y_center,w,h) 


for img in image_list:

    '''load image'''
    image = cv2.imread(data_dir + img)    
#     cv2.imshow('gt', image)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    
    '''get bounding box'''
    copy = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if not cnts:
        print(img + " is empty image")
        temp = sys.stdout
        with open(save_dir + img.strip('.jpg') + '.txt', 'w') as sys.stdout:
            print(0)
            sys.stdout = temp
        continue
        
    x_min,y_min,w,h = cv2.boundingRect(cnts[0])
    # ROI = image[y:y+h, x:x+w]
    # cv2.imwrite('p1_box.jpg', ROI)

    '''draw bounding box on image'''
    x_max = x_min + w
    y_max = y_min + h
    cv2.rectangle(copy,(x_min,y_min),(x_max,y_max),(36,255,12),2)
    cv2.imwrite(save_dir + img, copy)                    # save bounding box image

    '''convert bounding box coordinates to yolo style'''
    b = (x_min, y_min, x_max, y_max)                     # natural bounding box coordinates   
#     print('natural coordinates: ', b)

    bb = convert((copy.shape[1], copy.shape[0]), b)      # yolo style bounding box coordinates
#     print('result: ', bb)
    temp = sys.stdout
    with open(save_dir + img.strip('.jpg') + '.txt', 'w') as sys.stdout:
        print(0, round(bb[0],6),round(bb[1],6),round(bb[2],6),round(bb[3],6))
        sys.stdout = temp
    
# '''show bounding box image'''
# cv2.imshow('bounding box', copy)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [ ]:
import os

root_dir = 'C:/Users/kseongwoo/datasets/Polyp/18_video/Seqs/'

"""테스트 데이터셋의 파일 이름 변경: n.ext -> m_n.ext"""

def change_filename(type):
    for i in range(1, 19):
        dir = root_dir + str(i) + '/' + type + '/'
        file_list = os.listdir(dir)
        for j in file_list:
            file_newname = str(i) + "_" + j
            os.rename(dir + j, dir + file_newname)
            
# change_filename("GT")
# change_filename("label")
# change_filename("Original")

"""데이터셋 폴더에서 yolov5 테스트셋 폴더로 파일 복사"""
import shutil

# copy_dir = 'C:/Users/kseongwoo/yolov5/data/polyp/test/images'
# copy_dir = 'C:/Users/kseongwoo/yolov5/data/polyp/test/labels'

def copy_file(type):
    for i in range(1, 19):
        dir = root_dir + str(i) + '/' + type + '/'
        file_list = os.listdir(dir)
    
        for j in file_list:
            shutil.copy(dir + j ,copy_dir)

# copy_file("Original")
# copy_file("label")

In [5]:
"""테스트 데이터셋에서 용종이 없는 사진 개수 세기"""
import os
import sys

root_dir = 'C:/Users/kseongwoo/yolov5/data/polyp/test/labels/'
label_list = os.listdir(root_dir)

count = 0
for label in label_list:
    txtFile = open(root_dir + label, 'r')
    readLabel = txtFile.readline()
    read = readLabel.split()
    if len(read) == 1:
        count += 1
        
print(count)

1930


In [32]:
"""테스트 데이터셋을 영상으로 만들기 위해 파일 이름을 수정: m_n.jpg -> 00n.jpg"""

import os

root_dir = 'C:/Users/kseongwoo/datasets/Polyp/18_video/SeqsVideo/'

for i in range(1, 19):
    dir = root_dir + str(i) + '/'
    file_list = os.listdir(dir)
    digit_all = len(str(len(file_list))) # 전체 이미지 개수의 자릿수
    digit_seq = len(str(i)) # 시퀀스 번호의 자릿수
    print("-------------seqeunce " + str(i) + "---------------")
    for j in file_list:
        j_temp = j[digit_seq + 1:] 
        j_num = j_temp.rstrip(".jpg") # 현재 이미지 번호
        digit_now = len(str(j_num)) # 현재 이미지 번호의 자릿수
        
        # 현재 숫자 자릿수와 전체 숫자 자릿수 비교 ex) 1 vs 4
        # 모자란 만큼(두 수의 차만큼) 파일 이름에 0추가 ex) 1 -> 0001
        zero_count = digit_all - digit_now # 추가해야 하는 0의 개수
        
        new_file_name = ""
        
        for k in range(zero_count):
            new_file_name += "0"
        new_file_name += str(j_num)
        new_file_name += ".jpg"
        
        os.rename(dir + j, dir + new_file_name)

-------------seqeunce 1---------------
-------------seqeunce 2---------------
-------------seqeunce 3---------------
-------------seqeunce 4---------------
-------------seqeunce 5---------------
-------------seqeunce 6---------------
-------------seqeunce 7---------------
-------------seqeunce 8---------------
-------------seqeunce 9---------------
-------------seqeunce 10---------------
-------------seqeunce 11---------------
-------------seqeunce 12---------------
-------------seqeunce 13---------------
-------------seqeunce 14---------------
-------------seqeunce 15---------------
-------------seqeunce 16---------------
-------------seqeunce 17---------------
-------------seqeunce 18---------------
